In [5]:
import json
from spotify_api import SpotifyAPI
from config import client_id, client_secret

In [6]:
spotify = SpotifyAPI(client_id, client_secret)

Find the playlist unique SpotifyID

In [7]:
spotify.search('Most%Popular%Songs', search_type = 'playlist')

{'playlists': {'href': 'https://api.spotify.com/v1/search?query=Most%25Popular%25Songs&type=playlist&offset=0&limit=20',
  'items': [{'collaborative': False,
    'description': 'Only songs with one-hundred MILLION hits (either on Spotify or Youtube) will be posted here! These are the songs you MUST KNOW!',
    'external_urls': {'spotify': 'https://open.spotify.com/playlist/4o8NBsWreC3OnKePUYw0dg'},
    'href': 'https://api.spotify.com/v1/playlists/4o8NBsWreC3OnKePUYw0dg',
    'id': '4o8NBsWreC3OnKePUYw0dg',
    'images': [{'height': None,
      'url': 'https://i.scdn.co/image/ab67706c0000bebb4734bb829d2fc39bcac78822',
      'width': None}],
    'name': 'Most Popular Songs of All Time',
    'owner': {'display_name': 'zurxor',
     'external_urls': {'spotify': 'https://open.spotify.com/user/zurxor'},
     'href': 'https://api.spotify.com/v1/users/zurxor',
     'id': 'zurxor',
     'type': 'user',
     'uri': 'spotify:user:zurxor'},
    'primary_color': None,
    'public': None,
    'snap

Build a JSON file with the following data:

<ul>
<li>artist_id</li>
<li>artist_name</li>
<li>album_id</li>
<li>album_name</li>
<li>album_popularity</li>
<li>genres [list]</li>
<li>album_url</li>
</ul>

In [ ]:
playlist_id = '4o8NBsWreC3OnKePUYw0dg'
dataset_name = "data_full.txt"
dataset_path = ".\\data\\"

data = {}
id = 0
offset = 0

while True:
    cnt = 0
    r = spotify.get_playlist_items(playlist_id,
                                   'items(track(name,artists(id),id,album(id)))',
                                   100,
                                   offset, 
                                   'playlists')
    
    for item in r['items']:
        data[id] = {}
        artist_id = item['track']['artists'][0]['id'] 
        album_id = item['track']['album']['id']   
        artist = spotify.get_artist(artist_id)
        album = spotify.get_album(album_id)

        data[id]['artist_id'] = artist_id
        data[id]['artist_name'] = artist['name']
        data[id]['album_id'] = album_id
        data[id]['album_name'] = album['name']
        data[id]['album_popularity'] = album['popularity']
        data[id]['genres'] = artist['genres']
        
        for image in album['images']: 
            if image['height'] == 640:
                data[id]['album_url'] = image['url']
    
        id+=1
        cnt+=1
        
    print(f"Fetched {id} tracks.")
    
    if cnt < 100:
        break
        
    offset += 100
        
with open(dataset_path + dataset_name, 'w') as file:
    json.dump(data, file)